In [1]:
import pandas as pd
import datetime
import locale
import os

import matplotlib.pyplot as plt

locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

'pt_BR.utf8'

In [59]:
def read_data(filename):
    df_results = pd.read_csv(filename)
    df_results = df_results.drop('datas',axis=1)
    return df_results

In [62]:
def calculate_betha(df_results):
    covBench = df_results.cov()
    var = df_results.var(axis=0)
    betha = covBench.iat[0,1]/var[1]

    return betha

In [171]:
def read_cdi(dt_start,dt_end):
    start_month = datetime.datetime.strptime(dt_start, "%Y-%m-%d").strftime("%b").upper()
    start_year = datetime.datetime.strptime(dt_start, "%Y-%m-%d").strftime("%Y")

    end_month = datetime.datetime.strptime(dt_end, "%Y-%m-%d").strftime("%b").upper()
    end_year = datetime.datetime.strptime(dt_end, "%Y-%m-%d").strftime("%Y")

    # print(end_month)
    cdi_hist = pd.read_csv('../../data/cdi_historico_1986-2022.csv', delimiter=';')
    rf_df = pd.DataFrame()

    if end_year != start_year:
        for ano in range(int(start_year),int(end_year)+1):
            print(ano)
            if ano != int(end_year) and ano != int(start_year): # ano intermediario (entre inicial e final)
                rf_df = pd.concat([rf_df,cdi_hist[str(ano)]], axis=0)
            elif ano == int(end_year):
                print('end year')
                month_number = cdi_hist.index[cdi_hist['MES'] == end_month].tolist()[0]
                for j in range(month_number):
                    rf_df = pd.concat([rf_df,cdi_hist[str(ano)].filter(items=[j],axis=0)], axis=0)
            elif ano == int(start_year):
                print('start year')
                month_number = cdi_hist.index[cdi_hist['MES'] == start_month].tolist()[0]
                for k in range(month_number-1,12):
                    rf_df = pd.concat([rf_df,cdi_hist[str(ano)].filter(items=[k],axis=0)], axis=0)
    else:
        start_month_number = cdi_hist.index[cdi_hist['MES'] == start_month].tolist()[0]
        end_month_number = cdi_hist.index[cdi_hist['MES'] == end_month].tolist()[0]
        for m in range(start_month_number-1,end_month_number):
            rf_df = pd.concat([rf_df,cdi_hist[str(end_year)].filter(items=[m],axis=0)], axis=0)

    return rf_df

In [174]:
test = read_cdi('2020-01-01','2021-03-01')
print(test)

2020
start year
2021
end year
           0
0   0.003766
1   0.002937
2   0.003384
3   0.002849
4   0.002358
5   0.002123
6   0.001943
7   0.001599
8   0.001570
9   0.001570
10  0.001495
11  0.001645
0   0.001495
1   0.001345


In [126]:
def calculate_sharpe(df_results,rf):
    r = df_results['variacao_carteira'].mean()
    std = df_results['variacao_carteira'].std()
    Sf = (r-rf)/std
    return Sf

In [140]:
rf = pd.read_csv('../../data/cdi_historico_1986-2022.csv', delimiter=';')
rf.head(12)

# test = rf.index[rf['MES'] == 'AGO'].tolist()[0]
# print(test)
# print(rf)

,MES,2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1995,1994,1993,1992,1991,1990,1989,1988,1987,1986
0,JAN,0.00732300,0.001495,0.003766,0.005430,0.005834,0.010846,0.010549,0.009293,0.008398,...,0.034750,0.432508,0.287760,0.290656,0.234589,0.682318,0.238733,0.172270,0.146533,---
1,FEV,0.00755000,0.001345,0.002937,0.004936,0.004649,0.008638,0.010015,0.008185,0.007827,...,0.032375,0.414587,0.285610,0.303188,0.087163,0.799383,0.193551,0.186479,0.222613,---
2,MAR,0.00927100,0.002011,0.003384,0.004688,0.005316,0.010504,0.011605,0.010361,0.007599,...,0.044082,0.469793,0.285377,0.252048,0.100085,0.363878,0.205637,0.169116,0.140031,0.00868100
3,ABR,0.00834300,0.002078,0.002849,0.005183,0.005175,0.007853,0.010545,0.009483,0.008155,...,0.042199,0.459683,0.303854,0.237539,0.124891,0.051225,0.114695,0.202253,0.166968,0.01370600
4,MAI,0.01034600,0.002703,0.002358,0.005430,0.005175,0.009256,0.011075,0.009838,0.008583,...,0.042704,0.481178,0.309266,0.229388,0.126410,0.059380,0.115949,0.186721,0.256921,0.01321000
5,JUN,0.01015300,0.003078,0.002123,0.004688,0.005175,0.008081,0.011605,0.010658,0.008174,...,0.040498,0.484307,0.318009,0.242424,0.106589,0.093824,0.273353,0.202212,0.188929,0.01566900
6,JUL,0.01034800,0.003556,0.001943,0.005678,0.005422,0.007972,0.011075,0.011773,0.009404,...,0.040089,0.066772,0.327868,0.263088,0.132639,0.143426,0.327570,0.244889,0.094692,0.02225000
7,AGO,0.01169400,0.004279,0.001599,0.005017,0.005669,0.008015,0.012136,0.011075,0.008595,...,0.038066,0.041637,0.347975,0.257284,0.159726,0.126502,0.354062,0.226789,0.084823,0.02895500
8,SET,0.01072000,0.004420,0.001570,0.004638,0.004681,0.006377,0.011075,0.011075,0.009006,...,0.032529,0.038464,0.374313,0.276467,0.200071,0.175000,0.383510,0.259858,0.087287,0.03300900
9,OUT,0.01020700,0.004860,0.001570,0.004793,0.005430,0.006431,0.010474,0.011077,0.009448,...,0.030566,0.036548,0.383865,0.281949,0.260957,0.219679,0.472409,0.294777,0.104133,0.03454900


In [143]:
t_start_date = ["2021-01-01","2020-01-01","2020-01-01","2019-01-01","2019-01-01","2019-01-01"]
t_end_date = ["2021-12-01","2020-12-01","2020-12-01","2019-12-01","2019-12-01","2019-12-01"]
v_start_date = ["2021-01-01","2021-01-01","2020-01-01","2021-01-01","2020-01-01","2020-01-01"]
v_end_date = ["2021-12-01","2021-12-01","2020-12-01","2021-12-01","2020-12-01","2020-12-01"]

# df_bethas = pd.DataFrame(columns=['dt_ini_treinamento','dt_fin_treinamento','dt_ini_validacao','dt_fin_validacao','modelo','betha'])
df_bethas = pd.DataFrame()
index = 0
for i in range(len(t_start_date)):
    train_folder = (f"T_{t_start_date[i]}_{t_end_date[i]}")
    validation_folder = (f"V_{v_start_date[i]}_{v_end_date[i]}")

    path = f'../{train_folder}/{validation_folder}'
    for file in os.listdir(path):
        if file.endswith(".csv"):
            model = file.split('.')[0]
            filename = f'{path}/{model}.csv'
            # print(filename)
            results = read_data(filename)
            betha = calculate_betha(results)
            rf_df = read_cdi(v_start_date[i],v_end_date[i])
            rf = rf_df.iloc[:,0].mean()
            sharpe = calculate_sharpe(results,rf)
            # print(sharpe)
            betha_dict = {
                'dt_ini_treinamento': t_start_date[i],
                'dt_fin_treinamento': t_end_date[i],
                'dt_ini_validacao': v_start_date[i],
                'dt_fin_validacao': v_end_date[i],
                'modelo': model,
                'betha': betha,
                'sharpe': sharpe
            }
            betha_df = pd.DataFrame(data=betha_dict,index=[index])
            df_bethas = pd.concat([df_bethas,betha_df])
            index += 1

In [144]:
# print(df_bethas)
df_bethas.head(19)

,dt_ini_treinamento,dt_fin_treinamento,dt_ini_validacao,dt_fin_validacao,modelo,betha,sharpe
0,2021-01-01,2021-12-01,2021-01-01,2021-12-01,DMINMAX,1.157787,-0.239050
1,2021-01-01,2021-12-01,2021-01-01,2021-12-01,MAD,1.218392,-0.360378
2,2021-01-01,2021-12-01,2021-01-01,2021-12-01,MADD,1.237976,-0.340404
3,2021-01-01,2021-12-01,2021-01-01,2021-12-01,MENS,1.000000,-1.006964
4,2021-01-01,2021-12-01,2021-01-01,2021-12-01,MEQ,1.209095,-0.402375
5,2021-01-01,2021-12-01,2021-01-01,2021-12-01,MINMAX,1.163087,-0.253788
6,2021-01-01,2021-12-01,2021-01-01,2021-12-01,MVE,1.110773,-0.794290
7,2020-01-01,2020-12-01,2021-01-01,2021-12-01,DMINMAX,1.134743,-0.441663
8,2020-01-01,2020-12-01,2021-01-01,2021-12-01,MAD,1.142764,-0.666728
9,2020-01-01,2020-12-01,2021-01-01,2021-12-01,MADD,1.172167,-0.471990
